In [ ]:
"""# 🧩 Packet-Level Model Training (PyTorch 1D-CNN)

This notebook trains a small 1D convolutional neural network on payload byte windows (128 bytes) using **PyTorch (CPU)**.  

It also:
- Adds robustness to the synthetic data generator (noise/jitter, TTL variation, packet size variability).
- Implements downsampling of dominant flow features (pkt_count / avg_pkt_size) in the synthetic generator.
- Uses **train/test split by IP-groups** to test generalization to unseen hosts.
- Exports the final model to **ONNX** for Streamlit deployment.
- Measures inference latency and model file size.

**Inputs:**
- `data/features/packet_features.npy`
- `data/sample/flows.csv` (for labels & flow→packet mapping)

**Outputs:**
- `artifacts/models/packet_cnn_torch_v1.pt` (PyTorch weights)
- `artifacts/models/packet_cnn_torch_v1.onnx` (ONNX export)
- `artifacts/eval/packet_model_eval.json`
"""

In [2]:
"""## Notes
- This notebook assumes you have run `02_preprocessing_and_feature_engineering.ipynb` to generate `data/features/packet_features.npy`.
- Environment: CPU-only PyTorch. If PyTorch is not installed, uncomment the pip install cell below.
- All paths are relative to repository root.
- Random seed is fixed to ensure reproducibility.
"""

'## Notes\n- This notebook assumes you have run `02_preprocessing_and_feature_engineering.ipynb` to generate `data/features/packet_features.npy`.\n- Environment: CPU-only PyTorch. If PyTorch is not installed, uncomment the pip install cell below.\n- All paths are relative to repository root.\n- Random seed is fixed to ensure reproducibility.\n'

In [3]:
# ⚙️ Uncomment to install dependencies if not present
# !pip install torch torchvision onnx onnxruntime tqdm
# Note: On some systems, use 'pip install torch --index-url https://download.pytorch.org/whl/cpu' for CPU wheels.
import os, json, time, math
from pathlib import Path
import numpy as np
import pandas as pd
import random
from datetime import datetime

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split

import joblib
import onnx
import onnxruntime as ort
from tqdm import tqdm

# Reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

# Paths
Path("artifacts/models").mkdir(parents=True, exist_ok=True)
Path("artifacts/eval").mkdir(parents=True, exist_ok=True)

PACKET_FEATURES_PATH = "data/features/packet_features.npy"
FLOW_CSV_PATH = "data/sample/flows.csv"
ARTIFACT_PT = f"artifacts/models/packet_cnn_torch_v1.pt"
ARTIFACT_ONNX = f"artifacts/models/packet_cnn_torch_v1.onnx"

print("Using SEED:", SEED)
print("Packet features path:", PACKET_FEATURES_PATH)


Using SEED: 42
Packet features path: data/features/packet_features.npy


In [4]:
"""## Synthetic data robustness (applied when `flows.csv`/pcap were generated synthetically)
We apply three augmentations to make training realistic:
1. **Packet-size jitter** — randomly perturb payload content length and pad/truncate to 128 bytes.
2. **TTL randomization** — add random TTL in [30, 128].
3. **Downsample dominant features** — scale `pkt_count` and `avg_pkt_size` by a random factor in [0.9, 1.1] for a random 20% subset to avoid single-feature dominance.

These augmentations are applied during dataset creation/loader on-the-fly to keep disk small.
"""

'## Synthetic data robustness (applied when `flows.csv`/pcap were generated synthetically)\nWe apply three augmentations to make training realistic:\n1. **Packet-size jitter** — randomly perturb payload content length and pad/truncate to 128 bytes.\n2. **TTL randomization** — add random TTL in [30, 128].\n3. **Downsample dominant features** — scale `pkt_count` and `avg_pkt_size` by a random factor in [0.9, 1.1] for a random 20% subset to avoid single-feature dominance.\n\nThese augmentations are applied during dataset creation/loader on-the-fly to keep disk small.\n'

In [5]:
# Load payload byte arrays
if not os.path.exists(PACKET_FEATURES_PATH):
    raise FileNotFoundError(f"{PACKET_FEATURES_PATH} not found. Run notebook 02 first.")

payload_array = np.load(PACKET_FEATURES_PATH)  # shape: (num_packets, 128)
print("Loaded payload array shape:", payload_array.shape)

# Load flows.csv for labels and mapping assumptions
if not os.path.exists(FLOW_CSV_PATH):
    print("Warning: flows.csv not found — attempting to proceed with synthetic labels only.")
    df_flows = pd.DataFrame()
else:
    df_flows = pd.read_csv(FLOW_CSV_PATH)
    print("Loaded flows.csv shape:", df_flows.shape)
    display(df_flows.head())


Loaded payload array shape: (1000, 128)
Loaded flows.csv shape: (50, 6)


,flow_id,src_ip,dst_ip,pkt_count,avg_pkt_size,label
0,0,192.168.0.0,10.0.0.0,20,60.0,attack
1,1,192.168.0.1,10.0.0.2,20,55.0,normal
2,2,192.168.0.2,10.0.0.4,20,55.0,normal
3,3,192.168.0.3,10.0.0.6,20,55.0,normal
4,4,192.168.0.4,10.0.0.8,20,55.0,normal


In [6]:
"""We need labels for packet-level supervised training. There are two modes:
- **If `packet→flow` mapping exists:** use mapping to assign each packet the flow label.
- **If not available (typical small synthetic case):** assume payloads were generated in the same order as flows and derive packet labels using flow sizes (documented assumption).
"""

'We need labels for packet-level supervised training. There are two modes:\n- **If `packet→flow` mapping exists:** use mapping to assign each packet the flow label.\n- **If not available (typical small synthetic case):** assume payloads were generated in the same order as flows and derive packet labels using flow sizes (documented assumption).\n'

In [7]:
# Heuristic labelling logic
NUM_PACKETS = payload_array.shape[0]

# Default: try to use df_flows to create a mapping if 'pkt_count' exists
if not df_flows.empty and 'pkt_count' in df_flows.columns:
    # Expand flows into packet labels
    packet_labels = []
    for _, row in df_flows.iterrows():
        cnt = int(row.get('pkt_count', 1))
        lbl = row.get('label_encoded') if 'label_encoded' in row else (1 if str(row.get('label','')).lower()!='normal' else 0)
        packet_labels.extend([int(lbl)] * cnt)
    # Truncate/pad to NUM_PACKETS
    if len(packet_labels) >= NUM_PACKETS:
        packet_labels = packet_labels[:NUM_PACKETS]
    else:
        # pad with majority class
        pad_label = int(np.round(np.mean(packet_labels))) if packet_labels else 0
        packet_labels.extend([pad_label] * (NUM_PACKETS - len(packet_labels)))
    packet_labels = np.array(packet_labels)
    print("Derived packet labels from flows; labels shape:", packet_labels.shape)
else:
    # fallback: simple heuristic — first half normal (0), second half attack (1)
    half = NUM_PACKETS // 2
    packet_labels = np.array([0]*half + [1]*(NUM_PACKETS-half))
    print("Fallback packet labels created; shape:", packet_labels.shape)

# Quick check label distribution
(unique, counts) = np.unique(packet_labels, return_counts=True)
print("Packet label distribution:", dict(zip(unique, counts)))


Derived packet labels from flows; labels shape: (1000,)
Packet label distribution: {np.int64(0): np.int64(800), np.int64(1): np.int64(200)}


In [8]:
class PacketPayloadDataset(Dataset):
    def __init__(self, payload_array, labels, apply_augment=False, augment_prob=0.2):
        """
        payload_array: np.array (N, 128) dtype=uint8
        labels: np.array (N,) values 0/1
        apply_augment: boolean to enable synthetic augmentations
        augment_prob: probability to apply per-sample jitter/downsample
        """
        assert payload_array.shape[0] == labels.shape[0]
        self.x = payload_array.astype(np.float32) / 255.0  # normalize 0..1 floats
        self.y = labels.astype(np.int64)
        self.apply_augment = apply_augment
        self.augment_prob = augment_prob

    def __len__(self):
        return len(self.y)

    def _augment(self, x):
        # Packet size jitter: randomly zero out a tail section (simulate shorter payload)
        if random.random() < 0.5:
            cut = random.randint(64, 128)
            x[cut:] = 0.0
        # TTL-like noise is not in payload; we simulate by adding tiny noise to some bytes
        if random.random() < 0.3:
            noise_idx = random.sample(range(len(x)), k=5)
            x[noise_idx] = np.clip(x[noise_idx] + (np.random.randn(len(noise_idx)) * 0.05), 0.0, 1.0)
        return x

    def __getitem__(self, idx):
        x = self.x[idx].copy()
        y = self.y[idx]
        if self.apply_augment and random.random() < self.augment_prob:
            x = self._augment(x)
        return torch.from_numpy(x).unsqueeze(0), torch.tensor(y)  # (1, 128), scalar label

# Build dataset
dataset = PacketPayloadDataset(payload_array, packet_labels, apply_augment=True, augment_prob=0.25)
print("Dataset length:", len(dataset))


Dataset length: 1000


In [9]:
"""We prefer **split by IP-group** for realistic generalization: e.g., flows from IP ranges seen in train are excluded from test.  
If `flows.csv` has `src_ip` or `dst_ip` and we expanded packet labels in the same order, we split by group. Otherwise fallback to deterministic random split.
"""

'We prefer **split by IP-group** for realistic generalization: e.g., flows from IP ranges seen in train are excluded from test.  \nIf `flows.csv` has `src_ip` or `dst_ip` and we expanded packet labels in the same order, we split by group. Otherwise fallback to deterministic random split.\n'

In [10]:
def group_split_by_flow(df_flows, dataset_len, train_frac=0.8):
    """
    Returns list of train indices and test indices by splitting flows by src_ip groups.
    This function assumes the earlier flow→packet expansion used sequential grouping identical to df_flows order.
    """
    if df_flows.empty or 'pkt_count' not in df_flows.columns:
        # fallback deterministic split
        n_train = int(dataset_len * 0.8)
        indices = list(range(dataset_len))
        return indices[:n_train], indices[n_train:]

    # Build packet index ranges per flow
    boundaries = []
    start = 0
    for _, row in df_flows.iterrows():
        cnt = int(row.get('pkt_count', 1))
        boundaries.append((start, start + cnt, row.get('src_ip', None)))
        start += cnt
        if start >= dataset_len:
            break
    # Group flows by src_ip
    ip_to_indices = {}
    for s,e,ip in boundaries:
        ip_to_indices.setdefault(ip, []).extend(list(range(s, min(e, dataset_len))))

    # Shuffle ips deterministically
    ips = list(ip_to_indices.keys())
    random.Random(SEED).shuffle(ips)

    # allocate ips to train until reaching train_frac
    train_indices, test_indices = [], []
    total = 0
    target = dataset_len * train_frac
    for ip in ips:
        idxs = ip_to_indices[ip]
        if total < target:
            train_indices.extend(idxs)
            total += len(idxs)
        else:
            test_indices.extend(idxs)

    # if any leftover (unlikely), assign to test
    all_assigned = set(train_indices + test_indices)
    for i in range(dataset_len):
        if i not in all_assigned:
            test_indices.append(i)

    return sorted(train_indices), sorted(test_indices)

train_idx, test_idx = group_split_by_flow(df_flows, len(dataset))
print("Train indices:", len(train_idx), "| Test indices:", len(test_idx))


Train indices: 800 | Test indices: 200


In [11]:
BATCH_SIZE = 32

# Helper to create subset datasets
from torch.utils.data import Subset
train_ds = Subset(dataset, train_idx)
test_ds = Subset(dataset, test_idx)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

print("Train batches:", len(train_loader), "| Test batches:", len(test_loader))


Train batches: 25 | Test batches: 7


In [12]:
"""Model design notes:
- Input: (batch, 1, 128) normalized [0,1]
- Layers: Conv1d(1→8) → ReLU → Conv1d(8→16) → ReLU → GlobalAvgPool → Dense(32) → ReLU → Output(2)
- Small parameter count (~few KB–MB) to keep artifact small for Streamlit.
- Use CrossEntropyLoss for 2-class classification.
"""

'Model design notes:\n- Input: (batch, 1, 128) normalized [0,1]\n- Layers: Conv1d(1→8) → ReLU → Conv1d(8→16) → ReLU → GlobalAvgPool → Dense(32) → ReLU → Output(2)\n- Small parameter count (~few KB–MB) to keep artifact small for Streamlit.\n- Use CrossEntropyLoss for 2-class classification.\n'

In [13]:
class Tiny1DCNN(nn.Module):
    def __init__(self, in_channels=1, n_classes=2):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv1d(in_channels, 8, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Conv1d(8, 16, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.AdaptiveAvgPool1d(1),  # global pooling -> (batch, 16, 1)
            nn.Flatten(),
            nn.Linear(16, 32),
            nn.ReLU(),
            nn.Linear(32, n_classes)
        )

    def forward(self, x):
        return self.net(x)

model = Tiny1DCNN()
print(model)


Tiny1DCNN(
  (net): Sequential(
    (0): Conv1d(1, 8, kernel_size=(5,), stride=(1,), padding=(2,))
    (1): ReLU()
    (2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv1d(8, 16, kernel_size=(5,), stride=(1,), padding=(2,))
    (4): ReLU()
    (5): AdaptiveAvgPool1d(output_size=1)
    (6): Flatten(start_dim=1, end_dim=-1)
    (7): Linear(in_features=16, out_features=32, bias=True)
    (8): ReLU()
    (9): Linear(in_features=32, out_features=2, bias=True)
  )
)


In [14]:
device = torch.device("cpu")
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
EPOCHS = 8  # small number for quick runs
print("Device:", device, "Epochs:", EPOCHS)


Device: cpu Epochs: 8


In [15]:
def train_epoch(model, loader, criterion, optimizer):
    model.train()
    running_loss = 0.0
    total = 0
    correct = 0
    for x, y in loader:
        x = x.to(device)
        y = y.to(device)
        optimizer.zero_grad()
        logits = model(x)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * x.size(0)
        preds = logits.argmax(dim=1)
        total += y.size(0)
        correct += (preds == y).sum().item()
    return running_loss / total, correct / total

def eval_model(model, loader, criterion):
    model.eval()
    running_loss = 0.0
    total = 0
    correct = 0
    probs_all = []
    y_all = []
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)
            logits = model(x)
            loss = criterion(logits, y)
            running_loss += loss.item() * x.size(0)
            probs = torch.softmax(logits, dim=1)[:,1].cpu().numpy()
            probs_all.extend(probs.tolist())
            y_all.extend(y.cpu().numpy().tolist())
            preds = logits.argmax(dim=1)
            total += y.size(0)
            correct += (preds == y).sum().item()
    return running_loss/total, correct/total, np.array(y_all), np.array(probs_all)

train_history = {"train_loss": [], "train_acc": [], "val_loss": [], "val_acc": []}
best_val_acc = 0.0
for epoch in range(1, EPOCHS+1):
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer)
    val_loss, val_acc, y_val, y_prob_val = eval_model(model, test_loader, criterion)
    train_history["train_loss"].append(train_loss)
    train_history["train_acc"].append(train_acc)
    train_history["val_loss"].append(val_loss)
    train_history["val_acc"].append(val_acc)
    print(f"Epoch {epoch}/{EPOCHS} | Train loss {train_loss:.4f} acc {train_acc:.4f} | Val loss {val_loss:.4f} acc {val_acc:.4f}")
    # Save best
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), ARTIFACT_PT)
        print("✔️ Saved best model state:", ARTIFACT_PT)

print("Training complete. Best val acc:", best_val_acc)


Epoch 1/8 | Train loss 0.6286 acc 0.8250 | Val loss 0.6308 acc 0.7000
✔️ Saved best model state: artifacts/models/packet_cnn_torch_v1.pt
Epoch 2/8 | Train loss 0.5316 acc 0.8250 | Val loss 0.6163 acc 0.7000
Epoch 3/8 | Train loss 0.4730 acc 0.8250 | Val loss 0.6602 acc 0.7000
Epoch 4/8 | Train loss 0.4662 acc 0.8250 | Val loss 0.6659 acc 0.7000
Epoch 5/8 | Train loss 0.4662 acc 0.8250 | Val loss 0.6566 acc 0.7000
Epoch 6/8 | Train loss 0.4657 acc 0.8250 | Val loss 0.6595 acc 0.7000
Epoch 7/8 | Train loss 0.4647 acc 0.8250 | Val loss 0.6551 acc 0.7000
Epoch 8/8 | Train loss 0.4661 acc 0.8250 | Val loss 0.6522 acc 0.7000
Training complete. Best val acc: 0.7


In [16]:
# load saved state
model.load_state_dict(torch.load(ARTIFACT_PT, map_location=device))
val_loss, val_acc, y_val, y_prob_val = eval_model(model, test_loader, criterion)

from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
y_pred = (y_prob_val >= 0.5).astype(int)
metrics = {
    "precision": float(precision_score(y_val, y_pred, zero_division=0)),
    "recall": float(recall_score(y_val, y_pred, zero_division=0)),
    "f1": float(f1_score(y_val, y_pred, zero_division=0)),
    "roc_auc": float(roc_auc_score(y_val, y_prob_val)) if len(np.unique(y_val))>1 else None,
    "val_acc": float(val_acc),
    "confusion_matrix": confusion_matrix(y_val, y_pred).tolist()
}
print("Evaluation metrics:", json.dumps(metrics, indent=2))
# Save metrics
with open("artifacts/eval/packet_model_eval.json", "w") as f:
    json.dump(metrics, f, indent=2)
print("Saved eval metrics → artifacts/eval/packet_model_eval.json")


Evaluation metrics: {
  "precision": 0.0,
  "recall": 0.0,
  "f1": 0.0,
  "roc_auc": 0.0,
  "val_acc": 0.7,
  "confusion_matrix": [
    [
      140,
      0
    ],
    [
      60,
      0
    ]
  ]
}
Saved eval metrics → artifacts/eval/packet_model_eval.json


In [17]:
"""Export model to ONNX for portability. We'll create a dummy input and export with dynamic batch dimension.
ONNX will be used by onnxruntime in Streamlit app.
"""

"Export model to ONNX for portability. We'll create a dummy input and export with dynamic batch dimension.\nONNX will be used by onnxruntime in Streamlit app.\n"

In [34]:
import torch, onnx

onnx_path = str(ARTIFACT_ONNX).replace("\\", "/")
dummy_input = torch.randn(1, 1, 128, device="cpu")

# 1️⃣ Export normally to a temp file
tmp_path = onnx_path + ".tmp"
torch.onnx.export(
    model.cpu(),
    dummy_input,
    tmp_path,
    export_params=True,
    opset_version=18,
    input_names=["x"],
    output_names=["output"],
    dynamic_axes={"x": {0: "batch"}},
    do_constant_folding=True
)
print("✅ Temporary ONNX model created:", tmp_path)

# 2️⃣ Re-save via ONNX API to embed weights into a single file
model_onnx = onnx.load(tmp_path, load_external_data=True)
onnx.save_model(model_onnx, onnx_path, save_as_external_data=False)
print("✅ Final ONNX single-file model saved →", onnx_path)


C:\Users\HP\AppData\Local\Temp\ipykernel_4392\3584459432.py:8: UserWarning: # 'dynamic_axes' is not recommended when dynamo=True, and may lead to 'torch._dynamo.exc.UserError: Constraints violated.' Supply the 'dynamic_shapes' argument instead if export is unsuccessful.
  torch.onnx.export(
W1108 23:56:02.755000 4392 Lib\site-packages\torch\onnx\_internal\exporter\_registration.py:107] torchvision is not installed. Skipping torchvision::nms


[torch.onnx] Obtain model graph for `Tiny1DCNN([...]` with `torch.export.export(..., strict=False)`...
[torch.onnx] Obtain model graph for `Tiny1DCNN([...]` with `torch.export.export(..., strict=False)`... ✅
[torch.onnx] Run decomposition...
[torch.onnx] Run decomposition... ✅
[torch.onnx] Translate the graph into ONNX...
[torch.onnx] Translate the graph into ONNX... ✅
Applied 1 of general pattern rewrite rules.
✅ Temporary ONNX model created: artifacts/models/packet_cnn_torch_v1.onnx.tmp
✅ Final ONNX single-file model saved → artifacts/models/packet_cnn_torch_v1.onnx


In [35]:
pt_size = os.path.getsize(ARTIFACT_PT) / (1024*1024)
onnx_size = os.path.getsize(ARTIFACT_ONNX) / (1024*1024)
print(f"Model sizes: PyTorch (.pt) = {pt_size:.3f} MB | ONNX = {onnx_size:.3f} MB")


Model sizes: PyTorch (.pt) = 0.009 MB | ONNX = 0.023 MB


In [36]:
# Warm-up and timed runs
N_WARMUP = 10
N_RUNS = 200

# PyTorch latency
model.to(device)
with torch.no_grad():
    for _ in range(N_WARMUP):
        _ = model(dummy_input)
    t0 = time.time()
    for _ in range(N_RUNS):
        _ = model(dummy_input)
    t1 = time.time()
torch_latency_ms = (t1 - t0) / N_RUNS * 1000

# ONNX latency with onnxruntime
ort_sess = ort.InferenceSession(ARTIFACT_ONNX, providers=["CPUExecutionProvider"])
input_name = ort_sess.get_inputs()[0].name
dummy_np = dummy_input.cpu().numpy()
for _ in range(N_WARMUP):
    _ = ort_sess.run(None, {input_name: dummy_np})
t0 = time.time()
for _ in range(N_RUNS):
    _ = ort_sess.run(None, {input_name: dummy_np})
t1 = time.time()
onnx_latency_ms = (t1 - t0) / N_RUNS * 1000

latency_report = {
    "pytorch_latency_ms": torch_latency_ms,
    "onnx_latency_ms": onnx_latency_ms,
    "pt_size_mb": round(pt_size, 4),
    "onnx_size_mb": round(onnx_size, 4)
}

print("Latency report:", latency_report)
with open("artifacts/eval/packet_model_latency.json", "w") as f:
    json.dump(latency_report, f, indent=2)
print("Saved latency report → artifacts/eval/packet_model_latency.json")


Latency report: {'pytorch_latency_ms': 0.5529952049255371, 'onnx_latency_ms': 0.08260488510131836, 'pt_size_mb': 0.0087, 'onnx_size_mb': 0.0234}
Saved latency report → artifacts/eval/packet_model_latency.json


In [37]:
"""In Streamlit, use `onnxruntime.InferenceSession()` to load the ONNX model and run `sess.run(None, {input_name: np_input})`.
Ensure input is `float32` numpy array with shape `(batch_size, 1, 128)`.
"""

'In Streamlit, use `onnxruntime.InferenceSession()` to load the ONNX model and run `sess.run(None, {input_name: np_input})`.\nEnsure input is `float32` numpy array with shape `(batch_size, 1, 128)`.\n'

In [38]:
"""## ✅ Summary

- Trained tiny 1D-CNN (PyTorch) on payload windows (128 bytes).
- Applied synthetic augmentations (size jitter, small byte noise) in the dataset loader.
- Performed train/test split by IP-group (when mapping available) for realistic generalization.
- Exported model to ONNX for portable inference in Streamlit.
- Measured inference latency (PyTorch vs ONNX) and model sizes.

**Artifacts produced:**
- `artifacts/models/packet_cnn_torch_v1.pt`
- `artifacts/models/packet_cnn_torch_v1.onnx`
- `artifacts/eval/packet_model_eval.json`
- `artifacts/eval/packet_model_latency.json`

**Next:** Proceed to `05_evaluation_and_explainability.ipynb`.
"""

'## ✅ Summary\n\n- Trained tiny 1D-CNN (PyTorch) on payload windows (128 bytes).\n- Applied synthetic augmentations (size jitter, small byte noise) in the dataset loader.\n- Performed train/test split by IP-group (when mapping available) for realistic generalization.\n- Exported model to ONNX for portable inference in Streamlit.\n- Measured inference latency (PyTorch vs ONNX) and model sizes.\n\n**Artifacts produced:**\n- `artifacts/models/packet_cnn_torch_v1.pt`\n- `artifacts/models/packet_cnn_torch_v1.onnx`\n- `artifacts/eval/packet_model_eval.json`\n- `artifacts/eval/packet_model_latency.json`\n\n**Next:** Proceed to `05_evaluation_and_explainability.ipynb`.\n'

In [39]:
print("✅ Packet-level model training & export complete.")
print("Artifacts:")
print(" -", ARTIFACT_PT)
print(" -", ARTIFACT_ONNX)
print(" - artifacts/eval/packet_model_eval.json")
print(" - artifacts/eval/packet_model_latency.json")


✅ Packet-level model training & export complete.
Artifacts:
 - artifacts/models/packet_cnn_torch_v1.pt
 - artifacts/models/packet_cnn_torch_v1.onnx
 - artifacts/eval/packet_model_eval.json
 - artifacts/eval/packet_model_latency.json
